In [24]:
import exercice_extractor as ee

In [25]:
exos = ee.extract_exercices("sample.docx")

In [6]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords


In [1]:
from nltk.corpus import names

In [2]:
labeled_names = [(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')]
import random
random.shuffle(labeled_names)

In [4]:
def gender_features(word):
    return {'last_letter': word[-1]}

In [7]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [10]:
classifier.classify(gender_features("Neo"))

'male'

In [11]:
classifier.classify(gender_features("Lea"))

'female'

In [12]:
classifier.classify(gender_features("Gedeon"))

'male'

In [13]:
classifier.classify(gender_features("Elom"))

'male'

In [14]:
classifier.classify(gender_features("Jorjinho"))

'male'

In [15]:
nltk.classify.accuracy(classifier, test_set)

0.73

In [16]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     34.1 : 1.0
             last_letter = 'k'              male : female =     32.1 : 1.0
             last_letter = 'f'              male : female =     16.8 : 1.0
             last_letter = 'p'              male : female =     12.0 : 1.0
             last_letter = 'v'              male : female =     11.3 : 1.0


In [17]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

In [18]:
train_set

[({'last_letter': 'i'}, 'female'), ({'last_letter': 'e'}, 'female'), ...]

In [19]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [21]:
gender_features2("John")

{'first_letter': 'j',
 'last_letter': 'n',
 'count(a)': 0,
 'has(a)': False,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 0,
 'has(c)': False,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 0,
 'has(e)': False,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 1,
 'has(h)': True,
 'count(i)': 0,
 'has(i)': False,
 'count(j)': 1,
 'has(j)': True,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 0,
 'has(l)': False,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 1,
 'has(n)': True,
 'count(o)': 1,
 'has(o)': True,
 'count(p)': 0,
 'has(p)': False,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 0,
 'has(s)': False,
 'count(t)': 0,
 'has(t)': False,
 'count(u)': 0,
 'has(u)': False,
 'count(v)': 0,
 'has(v)': False,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

In [22]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.746


In [23]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

In [25]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.765


In [26]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))

In [27]:
for (tag, guess, name) in sorted(errors):
    print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Abigael                       
correct=female   guess=male     name=Addis                         
correct=female   guess=male     name=Adrien                        
correct=female   guess=male     name=Agnes                         
correct=female   guess=male     name=Alisun                        
correct=female   guess=male     name=Alleen                        
correct=female   guess=male     name=Allyn                         
correct=female   guess=male     name=Ambur                         
correct=female   guess=male     name=Anabel                        
correct=female   guess=male     name=Anais                         
correct=female   guess=male     name=Anet                          
correct=female   guess=male     name=Angel                         
correct=female   guess=male     name=Arlyn                         
correct=female   guess=male     name=Avis                          
correct=female   guess=male     name=Avril      

In [28]:
# Document classification
from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

In [29]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [32]:
print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

{'contains(plot)': True, 'contains(:)': True, 'contains(two)': True, 'contains(teen)': False, 'contains(couples)': False, 'contains(go)': False, 'contains(to)': True, 'contains(a)': True, 'contains(church)': False, 'contains(party)': False, 'contains(,)': True, 'contains(drink)': False, 'contains(and)': True, 'contains(then)': True, 'contains(drive)': False, 'contains(.)': True, 'contains(they)': True, 'contains(get)': True, 'contains(into)': True, 'contains(an)': True, 'contains(accident)': False, 'contains(one)': True, 'contains(of)': True, 'contains(the)': True, 'contains(guys)': False, 'contains(dies)': False, 'contains(but)': True, 'contains(his)': True, 'contains(girlfriend)': True, 'contains(continues)': False, 'contains(see)': False, 'contains(him)': True, 'contains(in)': True, 'contains(her)': False, 'contains(life)': False, 'contains(has)': True, 'contains(nightmares)': False, 'contains(what)': True, "contains(')": True, 'contains(s)': True, 'contains(deal)': False, 'contains

In [33]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [34]:
print(nltk.classify.accuracy(classifier, test_set))

0.83


In [35]:
classifier.show_most_informative_features(5)

Most Informative Features
    contains(schumacher) = True              neg : pos    =     10.6 : 1.0
 contains(unimaginative) = True              neg : pos    =      8.6 : 1.0
          contains(mena) = True              neg : pos    =      7.2 : 1.0
        contains(suvari) = True              neg : pos    =      7.2 : 1.0
     contains(atrocious) = True              neg : pos    =      6.8 : 1.0


In [36]:
#Part-of-Speech Tagging
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [37]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes)

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


In [38]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

In [39]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]

In [40]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]

In [41]:
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.6270512182993535

In [42]:
classifier.classify(pos_features('cats'))

'NNS'

In [43]:
def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]

In [44]:
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"), ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)
print(result)

(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


In [45]:
result.draw()

In [46]:
cp = nltk.RegexpParser('CHUNK: {<V.*> <TO> <V.*>}')
brown = nltk.corpus.brown
for sent in brown.tagged_sents():
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'CHUNK': print(subtree)

(CHUNK combined/VBN to/TO achieve/VB)
(CHUNK continue/VB to/TO place/VB)
(CHUNK serve/VB to/TO protect/VB)
(CHUNK wanted/VBD to/TO wait/VB)
(CHUNK allowed/VBN to/TO place/VB)
(CHUNK expected/VBN to/TO become/VB)
(CHUNK expected/VBN to/TO approve/VB)
(CHUNK expected/VBN to/TO make/VB)
(CHUNK intends/VBZ to/TO make/VB)
(CHUNK seek/VB to/TO set/VB)
(CHUNK like/VB to/TO see/VB)
(CHUNK designed/VBN to/TO provide/VB)
(CHUNK get/VB to/TO hear/VB)
(CHUNK expects/VBZ to/TO tell/VB)
(CHUNK expected/VBN to/TO give/VB)
(CHUNK prefer/VB to/TO pay/VB)
(CHUNK required/VBN to/TO obtain/VB)
(CHUNK permitted/VBN to/TO teach/VB)
(CHUNK designed/VBN to/TO reduce/VB)
(CHUNK Asked/VBN to/TO elaborate/VB)
(CHUNK got/VBN to/TO go/VB)
(CHUNK raised/VBN to/TO pay/VB)
(CHUNK scheduled/VBN to/TO go/VB)
(CHUNK cut/VBN to/TO meet/VB)
(CHUNK needed/VBN to/TO meet/VB)
(CHUNK hastened/VBD to/TO add/VB)
(CHUNK found/VBN to/TO prevent/VB)
(CHUNK continue/VB to/TO insist/VB)
(CHUNK compelled/VBN to/TO make/VB)
(CHUNK mad

(CHUNK seem/VB to/TO feel/VB)
(CHUNK decided/VBN to/TO keep/VB)
(CHUNK applied/VBN to/TO serve/VB)
(CHUNK try/VB to/TO seize/VB)
(CHUNK lead/VB to/TO involving/VBG)
(CHUNK cut/VBN to/TO allow/VB)
(CHUNK hope/VB to/TO enjoy/VB)
(CHUNK likes/VBZ to/TO receive/VB)
(CHUNK delighted/VBN to/TO learn/VB)
(CHUNK going/VBG to/TO expand/VB)
(CHUNK authorized/VBN to/TO deliver/VB)
(CHUNK going/VBG to/TO can/VB)
(CHUNK going/VBG to/TO close/VB)
(CHUNK supposed/VBN to/TO make/VB)
(CHUNK try/VB to/TO raise/VB)
(CHUNK tried/VBD to/TO settle/VB)
(CHUNK pass/VB to/TO enter/VB)
(CHUNK came/VBD to/TO tell/VB)
(CHUNK wanted/VBD to/TO grab/VB)
(CHUNK stops/VBZ to/TO think/VB)
(CHUNK expect/VB to/TO get/VB)
(CHUNK chosen/VBN to/TO answer/VB)
(CHUNK motivated/VBN to/TO seek/VB)
(CHUNK disposed/VBD to/TO covet/VB)
(CHUNK like/VB to/TO lie/VB)
(CHUNK going/VBG to/TO tell/VB)
(CHUNK prepared/VBN to/TO accept/VB)
(CHUNK try/VB to/TO stop/VB)
(CHUNK try/VB to/TO stop/VB)
(CHUNK sought/VBD to/TO break/VB)
(CHUNK i

(CHUNK continue/VB to/TO demonstrate/VB)
(CHUNK begins/VBZ to/TO wilt/VB)
(CHUNK going/VBG to/TO murder/VB)
(CHUNK trying/VBG to/TO puzzle/VB)
(CHUNK expects/VBZ to/TO profit/VB)
(CHUNK chooses/VBZ to/TO die/VB)
(CHUNK wishes/VBZ to/TO pay/VB)
(CHUNK conditioned/VBN to/TO expect/VB)
(CHUNK hoped/VBD to/TO tell/VB)
(CHUNK refusing/VBG to/TO make/VB)
(CHUNK intended/VBD to/TO stop/VB)
(CHUNK failed/VBD to/TO lead/VB)
(CHUNK need/VB to/TO discipline/VB)
(CHUNK attempts/VBZ to/TO trace/VB)
(CHUNK conspire/VB to/TO defy/VB)
(CHUNK learning/VBG to/TO think/VB)
(CHUNK wants/VBZ to/TO know/VB)
(CHUNK wants/VBZ to/TO supply/VB)
(CHUNK begin/VB to/TO look/VB)
(CHUNK start/VB to/TO think/VB)
(CHUNK attempt/VB to/TO say/VB)
(CHUNK come/VBN to/TO feel/VB)
(CHUNK supposed/VBN to/TO put/VB)
(CHUNK called/VBN to/TO work/VB)
(CHUNK continue/VB to/TO exist/VB)
(CHUNK observed/VBN to/TO characterize/VB)
(CHUNK continue/VB to/TO alienate/VB)
(CHUNK continue/VB to/TO preach/VB)
(CHUNK means/VBZ to/TO say/V

(CHUNK plan/VB to/TO increase/VB)
(CHUNK hopes/VBZ to/TO cut/VB)
(CHUNK bound/VBN to/TO swell/VB)
(CHUNK begin/VB to/TO feel/VB)
(CHUNK want/VB to/TO change/VB)
(CHUNK arrange/VB to/TO rent/VB)
(CHUNK plan/VB to/TO travel/VB)
(CHUNK plan/VB to/TO visit/VB)
(CHUNK like/VB to/TO start/VB)
(CHUNK wish/VB to/TO budget/VB)
(CHUNK wanting/VBG to/TO rent/VB)
(CHUNK want/VB to/TO see/VB)
(CHUNK want/VB to/TO throw/VB)
(CHUNK designed/VBN to/TO work/VB)
(CHUNK trying/VBG to/TO prepare/VB)
(CHUNK began/VBD to/TO develop/VB)
(CHUNK used/VBN to/TO suggest/VB)
(CHUNK wanted/VBD to/TO polish/VB)
(CHUNK pays/VBZ to/TO consider/VB)
(CHUNK compelled/VBN to/TO use/VB)
(CHUNK began/VBD to/TO fall/VB)
(CHUNK begun/VBN to/TO write/VB)
(CHUNK made/VBN to/TO seem/VB)
(CHUNK attempting/VBG to/TO direct/VB)
(CHUNK pleased/VBN to/TO call/VB)
(CHUNK scheduled/VBN to/TO nominate/VB)
(CHUNK come/VB to/TO spend/VB)
(CHUNK managed/VBD to/TO irrigate/VB)
(CHUNK stooped/VBD to/TO scoop/VB)
(CHUNK fall/VB to/TO show/VB

(CHUNK claimed/VBD to/TO own/VB)
(CHUNK tried/VBD to/TO find/VB)
(CHUNK meant/VBD to/TO pay/VB)
(CHUNK given/VBN to/TO go/VB)
(CHUNK wanted/VBD to/TO make/VB)
(CHUNK began/VBD to/TO gallop/VB)
(CHUNK attempting/VBG to/TO lasso/VB)
(CHUNK failing/VBG to/TO encircle/VB)
(CHUNK got/VBN to/TO get/VB)
(CHUNK trying/VBG to/TO maneuver/VB)
(CHUNK beginning/VBG to/TO go/VB)
(CHUNK began/VBD to/TO pull/VB)
(CHUNK began/VBD to/TO draw/VB)
(CHUNK like/VB to/TO go/VB)
(CHUNK tend/VB to/TO compete/VB)
(CHUNK doing/VBG to/TO help/VB)
(CHUNK begins/VBZ to/TO see/VB)
(CHUNK Desiring/VBG to/TO fill/VB)
(CHUNK tends/VBZ to/TO look/VB)
(CHUNK try/VB to/TO match/VB)
(CHUNK played/VBN to/TO win/VB)
(CHUNK failed/VBD to/TO make/VB)
(CHUNK began/VBD to/TO involve/VB)
(CHUNK learning/VBG to/TO bunt/VB)
(CHUNK dared/VBD to/TO dream/VB)
(CHUNK dared/VBD to/TO taunt/VB)
(CHUNK refused/VBD to/TO come/VB)
(CHUNK undertook/VBD to/TO set/VB)
(CHUNK essayed/VBD to/TO down/VB)
(CHUNK began/VBD to/TO prosper/VB)
(CHUNK

...

(CHUNK began/VBD to/TO weep/VB)
(CHUNK began/VBD to/TO move/VB)
(CHUNK tried/VBD to/TO push/VB)
(CHUNK tried/VBD to/TO rescue/VB)
(CHUNK seemed/VBD to/TO hold/VB)
(CHUNK began/VBD to/TO think/VB)
(CHUNK strove/VBD to/TO think/VB)
(CHUNK run/VB to/TO tell/VB)
(CHUNK fail/VB to/TO hear/VB)
(CHUNK dared/VBD to/TO wait/VB)
(CHUNK dared/VBD to/TO pat/VB)
(CHUNK trying/VBG to/TO push/VB)
(CHUNK began/VBD to/TO whirl/VB)
(CHUNK started/VBD to/TO worry/VB)
(CHUNK tried/VBD to/TO push/VB)
(CHUNK wanted/VBD to/TO get/VB)
(CHUNK tryin/VBG to/TO fuck/VB)
(CHUNK tried/VBD to/TO stifle/VB)
(CHUNK seeking/VBG to/TO kill/VB)
(CHUNK failed/VBD to/TO check/VB)
(CHUNK tried/VBD to/TO shut/VB)
(CHUNK refuses/VBZ to/TO believe/VB)
(CHUNK begun/VBN to/TO study/VB)
(CHUNK amazed/VBN to/TO discover/VB)
(CHUNK appear/VB to/TO reject/VB)
(CHUNK trying/VBG to/TO write/VB)
(CHUNK want/VB to/TO weep/VB)
(CHUNK love/VB to/TO know/VB)
(CHUNK like/VB to/TO know/VB)
(CHUNK seem/VB to/TO think/VB)
(CHUNK striving/VBG t

(CHUNK forgotten/VBN to/TO turn/VB)
(CHUNK meant/VBN to/TO shut/VB)
(CHUNK want/VB to/TO leave/VB)
(CHUNK want/VB to/TO go/VB)
(CHUNK got/VBD to/TO remember/VB)
(CHUNK got/VBD to/TO put/VB)
(CHUNK threaten/VB to/TO call/VB)
(CHUNK need/VB to/TO take/VB)
(CHUNK going/VBG to/TO get/VB)
(CHUNK get/VB to/TO know/VB)
(CHUNK paused/VBD to/TO look/VB)
(CHUNK seemed/VBD to/TO remember/VB)
(CHUNK Happened/VBN to/TO hear/VB)
(CHUNK got/VBN to/TO get/VB)
(CHUNK begun/VBN to/TO tell/VB)
(CHUNK begun/VBN to/TO question/VB)
(CHUNK began/VBD to/TO doubt/VB)
(CHUNK wished/VBD to/TO frighten/VB)
(CHUNK need/VB to/TO break/VB)
(CHUNK wished/VBN to/TO make/VB)
(CHUNK tried/VBN to/TO give/VB)
(CHUNK got/VBN to/TO eat/VB)
(CHUNK attempt/VB to/TO frighten/VB)
(CHUNK trying/VBG to/TO find/VB)
(CHUNK want/VB to/TO talk/VB)
(CHUNK refusing/VBG to/TO bear/VB)
(CHUNK Try/VB to/TO find/VB)
(CHUNK want/VB to/TO go/VB)
(CHUNK like/VB to/TO make/VB)
(CHUNK going/VBG to/TO go/VB)
(CHUNK Remember/VB to/TO tell/VB)
(CH

(CHUNK bothered/VBD to/TO speak/VB)
(CHUNK attempted/VBD to/TO salvage/VB)
(CHUNK seemed/VBD to/TO indicate/VB)
(CHUNK attempted/VBD to/TO form/VB)
(CHUNK tried/VBD to/TO pick/VB)
(CHUNK appeared/VBD to/TO disapprove/VB)
(CHUNK expected/VBN to/TO find/VB)
(CHUNK decided/VBD to/TO see/VB)
(CHUNK go/VB to/TO look/VB)
(CHUNK wished/VBD to/TO prepare/VB)
(CHUNK hoping/VBG to/TO see/VB)
(CHUNK cease/VB to/TO feel/VB)
(CHUNK forced/VBN to/TO admit/VB)
(CHUNK expect/VB to/TO see/VB)
(CHUNK liked/VBN to/TO liberate/VB)
(CHUNK relieved/VBN to/TO see/VB)
(CHUNK try/VB to/TO thank/VB)
(CHUNK like/VB to/TO starve/VB)
(CHUNK hoped/VBD to/TO succeed/VB)
(CHUNK hated/VBD to/TO answer/VB)
(CHUNK got/VBN to/TO get/VB)
(CHUNK swung/VBD to/TO see/VB)
(CHUNK offered/VBN to/TO walk/VB)
(CHUNK wanted/VBD to/TO avoid/VB)
(CHUNK dared/VBN to/TO enter/VB)
(CHUNK failing/VBG to/TO understand/VB)
(CHUNK pause/VB to/TO consider/VB)
(CHUNK chose/VBD to/TO ignore/VB)
(CHUNK dare/VB to/TO face/VB)
(CHUNK continued/V

(CHUNK like/VB to/TO offer/VB)
(CHUNK want/VB to/TO say/VB)
(CHUNK wished/VBD to/TO see/VB)
(CHUNK overheard/VBN to/TO say/VB)
(CHUNK like/VB to/TO get/VB)
(CHUNK expected/VBN to/TO perform/VB)
(CHUNK going/VBG to/TO bring/VB)
(CHUNK seek/VB to/TO storm/VB)
(CHUNK used/VBN to/TO defend/VB)
(CHUNK shocked/VBN to/TO find/VB)
(CHUNK hesitate/VB to/TO speak/VB)
(CHUNK beginning/VBG to/TO study/VB)
(CHUNK grow/VB to/TO devote/VB)
(CHUNK wish/VB to/TO turn/VB)
(CHUNK going/VBG to/TO fail/VB)
(CHUNK wished/VBD to/TO change/VB)
(CHUNK wanted/VBD to/TO take/VB)
(CHUNK wanted/VBD to/TO bring/VB)
(CHUNK like/VB to/TO know/VB)
(CHUNK intend/VB to/TO marry/VB)
(CHUNK began/VBD to/TO talk/VB)
(CHUNK used/VBD to/TO play/VB)
(CHUNK trying/VBG to/TO get/VB)
(CHUNK happen/VB to/TO drive/VB)
(CHUNK hating/VBG to/TO get/VB)
(CHUNK try/VB to/TO walk/VB)
(CHUNK left/VBN to/TO believe/VB)
(CHUNK tried/VBD to/TO rest/VB)
(CHUNK dying/VBG to/TO defend/VB)
(CHUNK used/VBD to/TO kid/VB)
(CHUNK wanted/VBD to/TO p

(CHUNK seems/VBZ to/TO overtake/VB)
(CHUNK want/VB to/TO buy/VB)
